In [81]:
import urllib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np
import umap
import itertools

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider, Range1d
from bokeh.layouts import column
from bokeh.palettes import all_palettes


output_notebook()

Loading BokehJS ...

In [82]:
# define Enrichr library names here if you're using Enrichr libraries
# or your own library if you are using your own
# library should be a GMT file, IF you wish to color the plot based on a metadata field
# name of gene sets should contain metadata fields separated by commas
# for example: ZIKV,human Calu-3,12 hpi,10 MOI,GSE123456
all_libraries = ['mygenesets_up.gmt.txt']

In [83]:
# open Enrichr library from online
def get_Enrichr_library(library_index):
    # processes library data
    raw_library_data = []
    library_data = []

    # open Enrichr library from online
    '''
    with urllib.request.urlopen('https://amp.pharm.mssm.edu/Enrichr/geneSetLibrary?mode=text&libraryName=' + all_libraries[library_index]) as f:
        for line in f.readlines():
                raw_library_data.append(line.decode("utf-8").split("\t\t"))
    '''

    # OR
    # locally upload library, provide file path to library
    with open('/Users/jessiecheng/Documents/maayanlab/mygenesets_up.gmt.txt') as f:
        for line in f.readlines():
            raw_library_data.append(line.split("\t\t"))

    name = []
    gene_list = []
    
    # fills name and gene_list, my version (works on "standard" GMT file format)
    temp_gene_list = [] 
    for i in range(len(raw_library_data)):
        raw_row = raw_library_data[i][0].split("\t")
        j = 0 
        while j < len(raw_row):
            if (raw_row[j] == "\n"):
                raw_row.pop(j)
            else:
                a = raw_row[j].strip()
                if (a == ""):
                    raw_row.pop(j)
                else:
                    raw_row[j] = a
                    j = j+1
        name.append(raw_row[0])
        temp_gene_list += [raw_row[2:]]
    
    for n in temp_gene_list:
        gene_string = ""
        for m in range(len(n)):
            gene_string = gene_string + n[m]
            gene_string = gene_string.lower()
            if (m != len(n)-1):
                gene_string = gene_string + " "
        gene_list = gene_list + [gene_string]
                
    # fills name and gene_list, Skylar's version, works on Enrichr GMT file format
    '''
    for i in range(len(raw_library_data)):
        name += [raw_library_data[i][0]]
        raw_genes = raw_library_data[i][1].replace('\t', ' ')
        gene_list += [raw_genes[:-1]]
    '''
   
    library_data = [list(a) for a in zip(name, gene_list)]
    
    return library_data

In [84]:
library_data = get_Enrichr_library(0)

df = pd.DataFrame(data = library_data, columns = ['Name', 'Genes'])

gene_list = df['Genes']

tfidf_vectorizer = TfidfVectorizer(
    min_df=1, # says it will ignore genes that fall in under 1 gene set
    max_df=0.85, # says it will ignore genes that fall in over 85% of the gene sets
    max_features = 10000, # I think this means it only looks at 10000 genes but I'm not totally sure
    ngram_range=(1, 1) # says it will only look at individual genes (because order doesn't matter in our case)
)

tfidf = tfidf_vectorizer.fit_transform(gene_list)

# Save the feature names for later to create topic summaries
tfidf_fn = tfidf_vectorizer.get_feature_names()

# plot using UMAP after tfidf
reduce = umap.UMAP()
reduce.fit(tfidf)
embedding = reduce.transform(tfidf)

embedding = pd.DataFrame(embedding, columns=['x','y'])

colorbyfield = 1 # can be 0 or 1, 0 if you don't want to color by a field 
                 # post-NMF graph will then color based on NMF groups determined by algorithm
                 # 1 if you do

if colorbyfield == 0:
    hues = [0]*(df.shape[0])
    my_labels = ['All']*(df.shape[0])
else:
    hues = [] 
    my_labels = []
    tracker = []
    field_index = 0 # denotes field to color by 
    for i in range(df.shape[0]):
        name_fields = (df.iloc[i]['Name']).split(",")
        field_value = name_fields[field_index]
        if field_value not in tracker:
            tracker.append(field_value)
        hues.append(tracker.index(field_value))
        my_labels.append(field_value)
    
embedding['hue'] = hues
my_colors = [(all_palettes['Category20'][20]+all_palettes['Category20b'][20])[i] for i in embedding.hue]

source1 = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            alpha = [0.7] * embedding.shape[0],
            size = [7] * embedding.shape[0], # can make size of the circles smaller by reducing the 7 here
            gene_set = df['Name'],
            colors = my_colors,
            label = my_labels
        )
    )

# just a print check here to make sure it's working correctly (feel free to delete)
print('There are ', embedding.shape[0], ' gene sets in this visualization')

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset', 'save']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
print(df)
plot_emb.circle('x', 'y', size='size', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source1, fill_color='colors', legend_group='label', name="df")


output_file("pre-NMF up (colored by virus) from " + all_libraries[0] + ".html")
show(plot_emb)

There are  89  gene sets in this visualization
                                                 Name  \
0   "ZIKV up,human whole blood cells,https://amp.p...   
1   "ZIKV up,human NPCs,https://amp.pharm.mssm.edu...   
2   "ZIKV up,human NPCs,https://amp.pharm.mssm.edu...   
3   "ZIKV up,human NPCs,https://amp.pharm.mssm.edu...   
4   "ZIKV up,human NPCs],https://amp.pharm.mssm.ed...   
5   "ZIKV up,human NPCs],https://amp.pharm.mssm.ed...   
6   "ZIKV up,human HUH7.5.1,https://amp.pharm.mssm...   
7   "DENV up,human HUH7.5.1,https://amp.pharm.mssm...   
8   "ZIKV up,human MDMs,https://amp.pharm.mssm.edu...   
9   "ZIKV up,green monkey Vero, https://amp.pharm....   
10  "ZIKV up,green monkey Vero,https://amp.pharm.m...   
11  "ZIKV up,green monkey Vero,https://amp.pharm.m...   
12  "EBOV up,human blood cells,https://amp.pharm.m...   
13  "EBOV-GP up,human PMBCs,https://amp.pharm.mssm...   
14  "EBOV-GP up,human PMBCs,https://amp.pharm.mssm...   
15  "EBOV up,human 293T cells,https://amp

In [85]:
# Create and run NMF model

n_comp = 9 # this is the number of groups you are splitting your gene set into

nmf = NMF(
    n_components=n_comp, 
    max_iter=100000,
    alpha=0.0
)

W = nmf.fit_transform(tfidf)
H = nmf.components_
nmf_embedding = nmf.transform(tfidf)

In [86]:
reducer = umap.UMAP()
reducer.fit(W)
embedding = reducer.transform(W)

embedding = umap.UMAP().fit_transform(tfidf.todense())
embedding = pd.DataFrame(embedding, columns=['x','y'])

# this will color by the groups created during NMF, if user doesn't want to color by field
# it's useful to look at what the model considers to be a "group"
# can use it help adjust the number of components in the NMF model
if colorbyfield == 0:
    embedding['hue'] = nmf_embedding.argmax(axis = 1)
    my_colors = [(all_palettes['Category20'][20]+all_palettes['Category20b'][20])[i] for i in embedding.hue]

source = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            alpha = [0.7] * embedding.shape[0],
            size = [7] * embedding.shape[0],
            gene_set = df['Name'],
            colors = my_colors,
            label = my_labels
        )
    )

print(embedding.shape[0])

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset', 'save']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
plot_emb.circle('x', 'y', size='size', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, fill_color='colors', legend_group='label', name="df")

output_file("post-NMF up (colored by virus) from " + all_libraries[0] + ".html")
show(plot_emb)

89


In [87]:
# this will print out the most common genes in each group
# not super useful but kind of interesting
n_topics = n_comp
n_top_words = 15

print("Topics found via NMF:")
for topic_idx, topic in enumerate(nmf.components_):
    print("\nTopic {}:".format(topic_idx+1))
    print(" ".join(['[{}]'.format(tfidf_fn[i]) for i in topic.argsort()[:-n_top_words - 1:-1]]))
print()

Topics found via NMF:

Topic 1:
[cflar] [akap2] [trim34] [akirin2] [ctnnd1] [lilrb1] [lilrb4] [cd80] [c20orf24] [tor1b] [apobec3a] [il8] [pric285] [adprhl2] [mb21d1]

Topic 2:
[ppme1] [zfyve28] [ikbip] [slc35e1] [upf1] [slc23a2] [ttll4] [sirt7] [sec62] [cse1l] [map4] [fabp4] [inpp5f] [wbp5] [ykt6]

Topic 3:
[akap8] [ggnbp2] [dusp7] [mnt] [snai3] [zmiz1] [fbxo34] [sertad3] [rbm15] [slc16a6] [rgcc] [c15orf39] [mef2d] [rgpd6] [pnrc2]

Topic 4:
[hla] [as1] [helz2] [dusp16] [klf4] [rnd1] [ccl5] [samhd1] [irf1] [bbc3] [ifih1] [plekha4] [parp14] [mx1] [fst]

Topic 5:
[h2] [mt] [ly6a] [t22] [q7] [ifi47] [igtp] [gm4070] [gvin1] [irgm1] [irgm2] [k1] [gbp7] [psma7] [atp6v1g1]

Topic 6:
[nufip1] [zbtb32] [loc108168990] [srr] [pes1] [gm39011] [utf1] [zfp593] [ddx39] [e130012a19rik] [e130218i03rik] [sssca1] [tma16] [eef1e1] [slmo2]

Topic 7:
[mt] [timm8b] [atp9b] [hnrnpm] [serpinb5] [agr2] [cyp2c18] [baat] [c10orf99] [entpd7] [nmur2] [glrx2] [sfta2] [adgrf1] [loc105377924]

Topic 8:
[as1] [thap9] [k